# <font color = red >Important!</font>

### <font color = red >When you make a copy of this notebook, share/save it as "Public", not Private.</font>
### For this Project, you need to create your own copy and share it. It is the shared link/url that would be submitted in canvas. To do this, implement the following steps:

1. **Log In to Kaggle:** First, ensure you are logged into your Kaggle account. If you don’t have an account, you will need to create one.
2. **Find the Notebook:** Navigate to this notebook. You can find notebooks by browsing the "Code" section or by searching for specific topics or datasets.In this case, search for "CSIS -126_Project 2 - Cstick Fall Prediction"
3. **Copy the Notebook:** Once you are on the notebook page, look for the "Copy and Edit" button, usually located at the top right of the notebook. Clicking this button creates a copy of the notebook in your account, which you can then edit and modify.
4. **Edit Your Notebook:** copying, you’ll be taken to a new instance of the notebook under your account. Here, you can make any changes or additions to the notebook.
5. **Save Your Changes:** As you work, make sure to save your changes by clicking the "Save Version" button. This will also run all the code cells in the notebook.
6. **Submit your notebook:** To submit your notebook, simply copy the URL from your browser when viewing your notebook and paste it in the corresponding assignment in Canvas. 

# **Introduction**
## Objective:
This Project involves applying supervised learning techniques, specifically **Logistic Regression** and **Support Vector Machines** (SVM), to the C-stick Fall Prediction Dataset. You can read about the dataset [here](https://www.kaggle.com/datasets/laavanya/elderly-fall-prediction-and-detection). You will then perform predictions On the data using each model, then finally you will compare the performance of these two algorithms and interpret their efficacy in making predictions based on the data.

## Steps
This Project is divided into 2 parts: **Section 1** and **Section 2**. Section 1 is implemented for you, and it involves loading the data, loading the necessary libraries and the separation of the data in features (inputs/attributes/Predictor variables) and labels (output/target/Predicted variables). You will implement Section 2, and it involves the following tasks:
1. Split the data into training sets and test sets
2. Implement the SVM Model
3. Implement predictions on the SVM model using an accuracy metric
4. Implement the Log Reg Model
5. Implement predictions on the Log Reg Model an accuracy metric
6. Answer Questions on your work

# **Section 1**
### Load Libraries, Data and separate data into features and labels

In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/cStick.csv
/kaggle/input/IFIP-IoT_2021_cStick-Elderly-Fall-Prediction.pdf
/kaggle/input/readme.docx


In [2]:
# Read in Data
df_Cstick = pd.read_csv('/kaggle/input/cStick.csv')
df_Cstick

,Distance,Pressure,HRV,Sugar level,SpO2,Accelerometer,Decision
0,25.540,1.0,101.396,61.080,87.770,1.0,1
1,2.595,2.0,110.190,20.207,65.190,1.0,2
2,68.067,0.0,87.412,79.345,99.345,0.0,0
3,13.090,1.0,92.266,36.180,81.545,1.0,1
4,69.430,0.0,89.480,80.000,99.990,0.0,0
...,...,...,...,...,...,...,...
2034,5.655,2.0,116.310,162.242,71.310,1.0,2
2035,9.660,2.0,124.320,177.995,79.320,1.0,2
2036,15.220,1.0,93.828,40.440,82.610,1.0,1
2037,9.120,2.0,123.240,175.871,78.240,1.0,2


## Summary of labels:
Label = 'Decision' column
 
Label Values:
* 0 - No Fall detected
* 1 - Slip detected
*  2 - Definite fall

In [3]:
outcome_dict = {0:'No Fall detected',1:'Slip detected',2:'Definite fall'}
outcome_list = ['No Fall detected','Slip detected','Definite fall']
outcome_list

['No Fall detected', 'Slip detected', 'Definite fall']

## Summary of features
features ='Distance', 'Pressure', 'HRV', 'Sugar level', 'SpO2', 'Accelerometer'

In [4]:
feature_list = ['Distance', 'Pressure', 'HRV', 'Sugar level', 'SpO2', 'Accelerometer']
feature_list

['Distance', 'Pressure', 'HRV', 'Sugar level', 'SpO2', 'Accelerometer']

## Features and Labels

In [5]:
df_Cstick_all_cols = df_Cstick.columns
df_Cstick_all_cols

Index(['Distance', 'Pressure', 'HRV', 'Sugar level', 'SpO2', 'Accelerometer',
       'Decision '],
      dtype='object')

In [6]:
# convert to float (if not already float)
df_Cstick1 = df_Cstick.astype(float)

# convert to numpy arrary
C = df_Cstick1.to_numpy()

# Get features and labels
features = C[:,0:6]
labels = C[:,-1] # or C[:,6]

features

array([[ 25.54 ,   1.   , 101.396,  61.08 ,  87.77 ,   1.   ],
       [  2.595,   2.   , 110.19 ,  20.207,  65.19 ,   1.   ],
       [ 68.067,   0.   ,  87.412,  79.345,  99.345,   0.   ],
       ...,
       [ 15.22 ,   1.   ,  93.828,  40.44 ,  82.61 ,   1.   ],
       [  9.12 ,   2.   , 123.24 , 175.871,  78.24 ,   1.   ],
       [ 62.441,   0.   ,  78.876,  76.435,  96.435,   0.   ]])

In [7]:
# Scramble the order of all the rows in the data to remove (potential) dependence on on its ordering 
# This makes our model more generalizable/robust
indices = np.random.permutation(len(labels)) 

# we scramble both X and y, necessarily with the same permutation. Use the same indices arragangement( otherwise we are doooomed!!)
X = features[indices]
y = labels[indices]

# **Section 2**
### <font color = red>Your Turn: Implement the following tasks

## Training and testing data
### Split the data into training data and test data. Use test_size of 0.2

In [8]:
# Implement your code here

# Training and testing data
# Split the data into training data and test data. Use test_size of 0.2
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Display shapes of training and testing sets
print(f"Training features shape: {X_train.shape}")
print(f"Testing features shape: {X_test.shape}")
print(f"Training labels shape: {y_train.shape}")
print(f"Testing labels shape: {y_test.shape}")

Training features shape: (1631, 6)
Testing features shape: (408, 6)
Training labels shape: (1631,)
Testing labels shape: (408,)


## Modelling

1a. SVM
Create and fit your SVM Model to the data
### 1a. SVM
### Create and fit your SVM Model to the data

In [9]:
# Implement your code here

# SVM Model
# Create and fit the SVM model to the data
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score, classification_report

svm_model = SVC(kernel='linear', random_state=42)
svm_model.fit(X_train, y_train)

# Evaluate the SVM model
y_pred_svm = svm_model.predict(X_test)

print("SVM Accuracy Score:")
print(accuracy_score(y_test, y_pred_svm))

print("\nSVM Classification Report:")
print(classification_report(y_test, y_pred_svm, target_names=outcome_list))

SVM Accuracy Score:
1.0

SVM Classification Report:
                  precision    recall  f1-score   support

No Fall detected       1.00      1.00      1.00       137
   Slip detected       1.00      1.00      1.00       133
   Definite fall       1.00      1.00      1.00       138

        accuracy                           1.00       408
       macro avg       1.00      1.00      1.00       408
    weighted avg       1.00      1.00      1.00       408



### 1b. Prediction
### Implement predictions on how well your SVM model performs using the accuracy metric

In [10]:
# Implement your code here

# Prediction
# Implement predictions and evaluate performance using accuracy metric

# Predict using the SVM model
y_pred_svm = svm_model.predict(X_test)

# Evaluate the SVM model using accuracy metric
svm_accuracy = accuracy_score(y_test, y_pred_svm)

# Print the accuracy of the SVM model
print('The accuracy of the SVM Model is:', svm_accuracy)

print("\nSVM Classification Report:")
print(classification_report(y_test, y_pred_svm, target_names=outcome_list))

The accuracy of the SVM Model is: 1.0

SVM Classification Report:
                  precision    recall  f1-score   support

No Fall detected       1.00      1.00      1.00       137
   Slip detected       1.00      1.00      1.00       133
   Definite fall       1.00      1.00      1.00       138

        accuracy                           1.00       408
       macro avg       1.00      1.00      1.00       408
    weighted avg       1.00      1.00      1.00       408



### 2a. Logistic Regression
### Create and fit your Logistic Regression Model to the data

In [11]:
# Implement your code here

# Logistic Regression Model
# Create and fit the Logistic Regression model to the data
from sklearn.linear_model import LogisticRegression

log_reg_model = LogisticRegression(random_state=42, max_iter=10000)
log_reg_model.fit(X_train, y_train)

LogisticRegression(max_iter=10000, random_state=42)

### 2b. Prediction
### Implement predictions on how well your Logistic Regression model performs using the accuracy metric

In [12]:
# Implement your code here

# Prediction
# Implement predictions and evaluate performance using accuracy metric

# Predict using the Logistic Regression model
y_pred_log_reg = log_reg_model.predict(X_test)

# Evaluate the Logistic Regression model using accuracy metric
log_reg_accuracy = accuracy_score(y_test, y_pred_log_reg)

# Print the accuracy of the Logistic Regression model
print('The accuracy of the Logistic Regression Model is:', log_reg_accuracy)

print("\nLogistic Regression Classification Report:")
print(classification_report(y_test, y_pred_log_reg, target_names=outcome_list))

The accuracy of the Logistic Regression Model is: 1.0

Logistic Regression Classification Report:
                  precision    recall  f1-score   support

No Fall detected       1.00      1.00      1.00       137
   Slip detected       1.00      1.00      1.00       133
   Definite fall       1.00      1.00      1.00       138

        accuracy                           1.00       408
       macro avg       1.00      1.00      1.00       408
    weighted avg       1.00      1.00      1.00       408



## Questions
### 1. Which model had a better performance? <font color=green>[Answer here]</font>
### 2. Why do you think the model performed better? <font color=green>[Answer here]</font>
### 3. List 4 things that could be done to improve the performance of any of these models ? <font color=green>[Answer here]</font>

